<a href="https://colab.research.google.com/github/lauren-esser/Recommendation-System/blob/main/MovieRecSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation-System

**By:** Lauren Esser

# Business Understanding

*not finished* --> The goal of this project is to build a model that provides top 5 movie recommendations to a user, based on their ratings of other movies. The business value behind this recommendation system is that it keeps users engaged. If a user sits down to watch a movie, but doesn't know what to select, using a recommendation system can help select movies that the user will actually enjoy and keep them coming back for more. This increases use of the website or app which thus increases sales. 

# Abstract


<img src = 'https://www.vshsolutions.com/wp-content/uploads/2020/02/recommender-system-for-movie-recommendation.jpg'>

# Introduction


# Import and Connect

In [1]:
#set numpy random seed
import numpy as np
np.random.seed(1919)

In [2]:
#set tensorflow random seed
import tensorflow as tf
tf.random.set_seed(1919)

In [3]:
!pip install pandasql

In [51]:
#import needed libraries
import pandas as pd
import pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os, glob
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


In [5]:
#connect to google drive
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [6]:
#move to home directory
%cd ~
%cd ..

/root
/


In [7]:
#list files in DataSets
!ls '/gdrive/My Drive/Colab Notebooks/DataSets'

archive.zip		      IVE_tickbidask.txt  News_Category_Dataset_v2.json
chest-xray-pneumonia-jmi.zip  Megatable-2016.sql
dogs-vs-cats-sorted.zip       ml-latest-small


# Obtain

Data comes from Grouplens website found [here.](https://grouplens.org/datasets/movielens/latest/) GroupLens is a research group in the Department of Computer Science and Engineering at the University of Minnesota. For this notebook I chose to download the *Small* MovieLens dataset. Movielens is a non-commerical website that allows users to rate movies. This dataset includes 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. The last update to this dataset was September 2018.



**Table Information**:

*Ratings*:
 Each row represents one rating of one movie by one user. Lines are organized by userId, then by movieId. Ratings are on a 5-star scale with .5 star increments. Timestamp represents seconds since midnight UTC of Jan 1, 1970.

---

*Tags*:
 Tags are user-generated metadata about movies. Tags are a single word or short phrase in which the meaning of tag is determined by each user.Timestamps represent seconds since midnight UTC of Jan 1, 1970.

---

*Movies*: 
Movie titles are entered manually or imported from themovieddb.org, this means there could be errors or inconsistencies with movie titles. Genre selection comes from a curated list for the user to select from. 

---

*Links*:
 Links include the identifier number for imdbID and tmdbID.

 ---

In [8]:
# #import and assign text file
# source_folder = r'gdrive/My Drive/Colab Notebooks/DataSets/'
# file = glob.glob(source_folder+ 'ml-latest-small.zip', recursive = True)
# file = file[0]
# file

In [9]:
# #unzip data
# zip_path = file
# !cp '{zip_path}' .

# !unzip -q ml-latest-small.zip 
# !rm ml-latest-small.zip

In [10]:
#create movies dataframe
movies = pd.read_csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
#create tags dataframe
tags = pd.read_csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [12]:
#create ratings dataframe
ratings = pd.read_csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
#create links dataframe
links = pd.read_csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


# Scrub

Use SQL to merge tables into one.

In [14]:
pysqldf = lambda q: sqldf(q, globals())

In [15]:
q = """SELECT *
       FROM movies m 
       JOIN links l
       ON m.movieId == l.movieId
       JOIN ratings r
       ON l.movieId == r.movieId
       JOIN tags t
       ON r.movieId == t.movieId;"""


In [16]:
df = pysqldf(q)
df.head()

,movieId,title,genres,movieId,imdbId,tmdbId,userId,movieId,rating,timestamp,userId,movieId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0,1,1,4.0,964982703,336,1,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0,1,1,4.0,964982703,474,1,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0,1,1,4.0,964982703,567,1,fun,1525286013
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0,5,1,4.0,847434962,336,1,pixar,1139045764
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,114709,862.0,5,1,4.0,847434962,474,1,pixar,1137206825


In [17]:
#remove duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

In [18]:
#check nulls
df.isna().sum()

movieId      0
title        0
genres       0
imdbId       0
tmdbId       0
userId       0
rating       0
timestamp    0
tag          0
dtype: int64

In [39]:
df1 = df[['movieId', 'userId', 'rating']]
df1

,movieId,userId,rating
0,1,1,4.0
1,1,1,4.0
2,1,1,4.0
3,1,5,4.0
4,1,5,4.0
...,...,...,...
233208,187595,586,5.0
233209,193565,184,3.5
233210,193565,184,3.5
233211,193565,184,3.5


In [37]:
#basic information
print("Total Data:")
print("Total Number of Movie Ratings: " + str(df1.shape[0]))
print("Number of unique users: " + str(len(np.unique(df1["userId"]))))
print("Number of unique movies: " + str(len(np.unique(df1["movieId"]))))

Total Data:
Total Number of Movie Ratings: 233213
Number of unique users: 610
Number of unique movies: 1554


In [45]:
df1.to_csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/df.csv', index = False)

# Explore

In [23]:
!pip install pyspark
from pyspark.sql import SparkSession

In [24]:
#instantiate Spark
spark = SparkSession\
        .builder\
        .appName('ALSExample').config('spark.driver.host', 'localhost')\
        .getOrCreate()

In [46]:
#read in dataset into pyspark
movie = spark.read.csv('gdrive/My Drive/Colab Notebooks/DataSets/ml-latest-small/df.csv', header='true', inferSchema='true')

# Model

### Alternating Least Squares Model

In [47]:
movie.dtypes

[('movieId', 'int'), ('userId', 'int'), ('rating', 'double')]

In [48]:
#create training and test set
(training, test) = movie.randomSplit([0.8, 0.2])


In [49]:
#recommendation model with ALS
als = ALS(maxIter=5,rank=4, regParam=0.01, userCol='userId', itemCol='movieId', 
          ratingCol='rating', coldStartStrategy='drop')

# fit model to training
model = als.fit(training)

In [50]:
#compute RMSE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'rating',
                                predictionCol = 'prediction')
rmse = evaluator.evaluate(predictions)

print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 0.6456802890249984


---

**Cross Validation for Optimal Model**

In [ ]:
#instantiate vanilla model
als_model = ALS(userCol='userId', itemCol='movieId', 
          ratingCol='rating', coldStartStrategy='drop')

#create param grid
params = ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001, 0.1])\
          .addGrid(als_model.rank, [4, 10, 50])\
          .addGrid(als_model.maxIter, [5, 10]).build()

#instantiate crossval
cv = CrossValidator(estimator = als_model, estimatorParamMaps = params,
                    evaluator = evaluator, parallelism = 4)
best_model = cv.fit(movie)

best_model.bestModel.rank

# Interpret

# Recommendations

# Future Work

Thank you.